In [1]:
# Imports
import pandas as pd
import numpy as np
from pathlib import Path
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn import svm
from sklearn.preprocessing import StandardScaler
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report
from prophet import Prophet

/Users/davidrosenbloom/anaconda3/envs/dev/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Importing plotly failed. Interactive plots will not work.


In [2]:
#Read in the raw data from WorldBank
data_df = pd.read_csv(
    Path("./Resources/03c00fd0-1cf9-46e8-aa22-40e2b0adbc27_Data.csv"), 
)

data_df.head()

,Country Name,Country Code,Series Name,Series Code,1973 [YR1973],1974 [YR1974],1975 [YR1975],1976 [YR1976],1977 [YR1977],1978 [YR1978],...,2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020],2021 [YR2021],2022 [YR2022]
0,Argentina,ARG,"Adolescent fertility rate (births per 1,000 wo...",SP.ADO.TFRT,68.608,70.631,72.626,74.274,75.771,75.957,...,69.167,67.791,65.395,61.852,57.783,51.029,46.153,39.866,39.065,..
1,Argentina,ARG,"Agriculture, forestry, and fishing, value adde...",NV.AGR.TOTL.ZS,11.9525224881086,10.2308377488651,6.5839104917193,8.15217071197931,8.0871421139688,7.5044274778029,...,6.05291843670451,6.71270351428559,5.1566859021408,6.26456582010254,5.23162237725058,4.53787889681146,5.3185559967348,6.12846695022316,7.13154104672929,6.44416917103214
2,Argentina,ARG,"Annual freshwater withdrawals, total (% of int...",ER.H2O.FWTL.ZS,..,..,..,9.45205479452055,9.46970620044521,9.48735760633562,...,12.9075342465753,12.9075342465753,12.9075342465753,12.9075342465753,12.9075342465753,12.9075342465753,12.9075342465753,12.9075342465753,..,..
3,Argentina,ARG,Births attended by skilled health staff (% of ...,SH.STA.BRTC.ZS,..,..,..,..,..,..,...,97,99.6,99.6,98.4,93.9,99.5,99.6,98.8,..,..
4,Argentina,ARG,CO2 emissions (metric tons per capita),EN.ATM.CO2E.PC,..,..,..,..,..,..,...,4.34225013023383,4.20911189491323,4.30191380564475,4.20181586904703,4.07011168693629,3.97565074444479,3.74202981162433,3.4056175404138,..,..


In [3]:
# We don't need these two columns
data_df = data_df.drop(columns=['Country Name','Series Code']).copy()

In [4]:
# Group by country and then by series name so we can provide per-country analysis
grouped = data_df.groupby([
    'Country Code',
    'Series Name'
    ])

In [5]:
# We want to return all the data for the aggregation so just return the data during aggregation
grouped = grouped.agg(lambda x: x)

# Replace the default missing data value with NaN
grouped = grouped.replace('..', np.nan)

In [6]:
# Manually add dummy variables to allow us to train a model on a per country basis 
# while still including all information of the other countries
grouped_transpose = grouped.T.copy()

country_codes = list(set(grouped.index.get_level_values(0)))

number_of_codes = len(country_codes)

for code in country_codes:
    print(f"Remaining {number_of_codes}")
    for code_false in country_codes:
        grouped_transpose[code, f"code_{code_false}"] = code == code_false
    number_of_codes -= 1

grouped_transpose = grouped_transpose.sort_index(axis=1)

def convert_to_datetime(x):
    return x.strip().split(' ')[0]

index_dates = grouped_transpose.index.to_series().apply(convert_to_datetime)

grouped_transpose.index = pd.to_datetime(index_dates)

# This was already done before no need to do it again
# grouped_transpose = grouped_transpose.replace('..', np.nan)

Remaining 266
Remaining 265
Remaining 264


/var/folders/s7/5vbrzq7x741_dp271_qgj7wc0000gn/T/ipykernel_25416/2031278900.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_transpose[code, f"code_{code_false}"] = code == code_false
/var/folders/s7/5vbrzq7x741_dp271_qgj7wc0000gn/T/ipykernel_25416/2031278900.py:12: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  grouped_transpose[code, f"code_{code_false}"] = code == code_false
/var/folders/s7/5vbrzq7x741_dp271_qgj7wc0000gn/T/ipykernel_25416/2031278900.py:12: PerformanceWarning: DataFrame is highly fragmented.  This i

Remaining 263
Remaining 262
Remaining 261
Remaining 260
Remaining 259
Remaining 258
Remaining 257
Remaining 256
Remaining 255
Remaining 254
Remaining 253
Remaining 252
Remaining 251
Remaining 250
Remaining 249
Remaining 248
Remaining 247
Remaining 246
Remaining 245
Remaining 244
Remaining 243
Remaining 242
Remaining 241
Remaining 240
Remaining 239
Remaining 238
Remaining 237
Remaining 236
Remaining 235
Remaining 234
Remaining 233
Remaining 232
Remaining 231
Remaining 230
Remaining 229
Remaining 228
Remaining 227
Remaining 226
Remaining 225
Remaining 224
Remaining 223
Remaining 222
Remaining 221
Remaining 220
Remaining 219
Remaining 218
Remaining 217
Remaining 216
Remaining 215
Remaining 214
Remaining 213
Remaining 212
Remaining 211
Remaining 210
Remaining 209
Remaining 208
Remaining 207
Remaining 206
Remaining 205
Remaining 204
Remaining 203
Remaining 202
Remaining 201
Remaining 200
Remaining 199
Remaining 198
Remaining 197
Remaining 196
Remaining 195
Remaining 194
Remaining 193
Remain

In [7]:
# Save the output to JSON for repeat use
grouped_transpose.to_json(Path('processed.json'))

In [10]:
grouped_transpose['ABW']['Fertility rate, total (births per woman)']

1973-01-01    2.613
1974-01-01    2.552
1975-01-01    2.506
1976-01-01    2.472
1977-01-01    2.446
1978-01-01    2.425
1979-01-01    2.408
1980-01-01    2.392
1981-01-01    2.377
1982-01-01    2.364
1983-01-01    2.353
1984-01-01    2.342
1985-01-01    2.332
1986-01-01     2.32
1987-01-01    2.307
1988-01-01    2.291
1989-01-01    2.272
1990-01-01    2.303
1991-01-01    2.314
1992-01-01    2.277
1993-01-01    2.226
1994-01-01    2.125
1995-01-01     2.19
1996-01-01    2.153
1997-01-01    2.144
1998-01-01    1.957
1999-01-01    1.869
2000-01-01    1.904
2001-01-01    1.833
2002-01-01    1.763
2003-01-01    1.747
2004-01-01    1.683
2005-01-01    1.777
2006-01-01    1.909
2007-01-01    1.928
2008-01-01    1.935
2009-01-01    1.919
2010-01-01    1.941
2011-01-01    1.962
2012-01-01    2.028
2013-01-01    2.117
2014-01-01    2.148
2015-01-01    1.972
2016-01-01    1.953
2017-01-01    1.839
2018-01-01    1.587
2019-01-01    1.486
2020-01-01    1.325
2021-01-01     1.18
2022-01-01      NaN
